<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Box Plots**


Estimated time needed: **45** minutes


In this lab, you will focus on the visualization of data. The dataset will be provided through an RDBMS, and you will need to use SQL queries to extract the required data.


## Objectives


In this lab you will perform the following:


-   Visualize the distribution of data.

-   Visualize the relationship between two features.

-   Visualize data composition and comparisons using box plots.


### Setup: Connecting to the Database


#### 1. Download the Database File


In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/QR9YeprUYhOoLafzlLspAw/survey-results-public.sqlite

#### 2. Connect to the Database


**Install the needed libraries**


In [ ]:
!pip install pandas

In [ ]:
!pip install matplotlib

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

# Connect to the SQLite database
conn = sqlite3.connect('survey-results-public.sqlite')


## Demo: Basic SQL Queries


#### Demo 1: Count the Number of Rows in the Table


In [ ]:
QUERY = "SELECT COUNT(*) FROM main"
df = pd.read_sql_query(QUERY, conn)
print(df)


#### Demo 2: List All Tables


In [ ]:
QUERY = """
SELECT name as Table_Name 
FROM sqlite_master 
WHERE type = 'table'
"""
pd.read_sql_query(QUERY, conn)


#### Demo 3: Group Data by Age


In [ ]:
QUERY = """
SELECT Age, COUNT(*) as count 
FROM main 
GROUP BY Age 
ORDER BY Age
"""
df_age = pd.read_sql_query(QUERY, conn)
print(df_age)


## Visualizing Data


### Task 1: Visualizing the Distribution of Data


**1. Box Plot of `CompTotal` (Total Compensation)**


Use a box plot to analyze the distribution and outliers in total compensation.


In [ ]:
# your code goes here
QUERY = """
SELECT CompTotal
FROM main
WHERE CompTotal IS NOT NULL
"""
df = pd.read_sql_query(QUERY, conn)

df["CompTotal"] = pd.to_numeric(df["CompTotal"], errors="coerce")
df = df.dropna()

plt.figure(figsize=(8,6))
plt.boxplot(df["CompTotal"], vert=True)
plt.title("Box Plot of CompTotal")
plt.ylabel("CompTotal")
plt.show()

**2. Box Plot of Age (converted to numeric values)**


Convert the `Age` column into numerical values and visualize the distribution.


In [ ]:
# your code goes here
QUERY = """
SELECT Age
FROM main
WHERE Age IS NOT NULL
"""
df_age = pd.read_sql_query(QUERY, conn)

df_age["Age"] = pd.to_numeric(df_age["Age"], errors="coerce")
df_age = df_age.dropna()

plt.figure(figsize=(8,6))
plt.boxplot(df_age["Age"], vert=True)
plt.title("Box Plot of Age")
plt.ylabel("Age")
plt.show()

### Task 2: Visualizing Relationships in Data


**1. Box Plot of `CompTotal` Grouped by Age Groups:**


Visualize the distribution of compensation across different age groups.


In [ ]:
# your code goes here
QUERY = """
SELECT Age, CompTotal
FROM main
WHERE Age IS NOT NULL AND CompTotal IS NOT NULL
"""
df = pd.read_sql_query(QUERY, conn)

df["Age"] = pd.to_numeric(df["Age"], errors="coerce")
df["CompTotal"] = pd.to_numeric(df["CompTotal"], errors="coerce")
df = df.dropna()

young = df[df["Age"] < 30]["CompTotal"]
mid = df[(df["Age"] >= 30) & (df["Age"] < 50)]["CompTotal"]
senior = df[df["Age"] >= 50]["CompTotal"]

plt.figure(figsize=(8,6))
plt.boxplot([young, mid, senior], labels=["<30", "30-49", "50+"])
plt.title("CompTotal by Age Group")
plt.ylabel("CompTotal")
plt.show()

**2. Box Plot of `CompTotal` Grouped by Job Satisfaction (`JobSatPoints_6`):**


Examine how compensation varies based on job satisfaction levels.


In [ ]:
# your code goes here
QUERY = """
SELECT JobSatPoints_6, CompTotal
FROM main
WHERE JobSatPoints_6 IS NOT NULL AND CompTotal IS NOT NULL
"""
df = pd.read_sql_query(QUERY, conn)

df["JobSatPoints_6"] = pd.to_numeric(df["JobSatPoints_6"], errors="coerce")
df["CompTotal"] = pd.to_numeric(df["CompTotal"], errors="coerce")
df = df.dropna()

groups = []
labels = []

for val in sorted(df["JobSatPoints_6"].unique()):
    groups.append(df[df["JobSatPoints_6"] == val]["CompTotal"])
    labels.append(str(int(val)))

plt.figure(figsize=(10,6))
plt.boxplot(groups, labels=labels)
plt.title("CompTotal by Job Satisfaction Level")
plt.xlabel("JobSatPoints_6")
plt.ylabel("CompTotal")
plt.show()

### Task 3: Visualizing the Composition of Data


**1. Box Plot of `ConvertedCompYearly` for the Top 5 Developer Types:**


Analyze compensation across the top 5 developer roles.


In [ ]:
# your code goes here
QUERY = """
SELECT DevType, ConvertedCompYearly
FROM main
WHERE DevType IS NOT NULL AND ConvertedCompYearly IS NOT NULL
"""
df = pd.read_sql_query(QUERY, conn)

df["ConvertedCompYearly"] = pd.to_numeric(df["ConvertedCompYearly"], errors="coerce")
df = df.dropna()

# Split multiple dev types
df["DevType"] = df["DevType"].str.split(";")
df = df.explode("DevType")
top5 = df["DevType"].value_counts().head(5).index
groups = [df[df["DevType"] == dev]["ConvertedCompYearly"] for dev in top5]

plt.figure(figsize=(10,6))
plt.boxplot(groups, labels=top5)
plt.xticks(rotation=45)
plt.title("ConvertedCompYearly for Top 5 Developer Types")
plt.ylabel("ConvertedCompYearly")
plt.show()

**2. Box Plot of `CompTotal` for the Top 5 Countries:**


Analyze compensation across respondents from the top 5 countries.


In [ ]:
# your code goes here
QUERY = """
SELECT Country, CompTotal
FROM main
WHERE Country IS NOT NULL AND CompTotal IS NOT NULL
"""
df = pd.read_sql_query(QUERY, conn)
df["CompTotal"] = pd.to_numeric(df["CompTotal"], errors="coerce")
df = df.dropna()
top5 = df["Country"].value_counts().head(5).index
groups = [df[df["Country"] == c]["CompTotal"] for c in top5]

plt.figure(figsize=(10,6))
plt.boxplot(groups, labels=top5)
plt.xticks(rotation=45)
plt.title("CompTotal for Top 5 Countries")
plt.ylabel("CompTotal")
plt.show()

### Task 4: Visualizing Comparison of Data


**1. Box Plot of CompTotal Across Employment Types:**


Analyze compensation for different employment types.


In [ ]:
# your code goes here
QUERY = """
SELECT Employment, CompTotal
FROM main
WHERE Employment IS NOT NULL AND CompTotal IS NOT NULL
"""
df = pd.read_sql_query(QUERY, conn)
df["CompTotal"] = pd.to_numeric(df["CompTotal"], errors="coerce")
df = df.dropna()
top_emp = df["Employment"].value_counts().head(5).index
groups = [df[df["Employment"] == e]["CompTotal"] for e in top_emp]

plt.figure(figsize=(10,6))
plt.boxplot(groups, labels=top_emp)
plt.xticks(rotation=45)
plt.title("CompTotal by Employment Type")
plt.ylabel("CompTotal")
plt.show()

**2. Box Plot of `YearsCodePro` by Job Satisfaction (`JobSatPoints_6`):**


Examine the distribution of professional coding years by job satisfaction levels.


In [ ]:
# your code goes here
QUERY = """
SELECT YearsCodePro, JobSatPoints_6
FROM main
WHERE YearsCodePro IS NOT NULL AND JobSatPoints_6 IS NOT NULL
"""
df = pd.read_sql_query(QUERY, conn)
df["YearsCodePro"] = pd.to_numeric(df["YearsCodePro"], errors="coerce")
df["JobSatPoints_6"] = pd.to_numeric(df["JobSatPoints_6"], errors="coerce")
df = df.dropna()
groups = []
labels = []
for val in sorted(df["JobSatPoints_6"].unique()):
    groups.append(df[df["JobSatPoints_6"] == val]["YearsCodePro"])
    labels.append(str(int(val)))

plt.figure(figsize=(10,6))
plt.boxplot(groups, labels=labels)
plt.title("YearsCodePro by Job Satisfaction Level")
plt.xlabel("JobSatPoints_6")
plt.ylabel("YearsCodePro")
plt.show()

### Final Step: Close the Database Connection


After completing the lab, close the connection to the SQLite database:


In [ ]:
conn.close()

## Summary


In this lab, you used box plots to visualize various aspects of the dataset, focusing on:

- Visualize distributions of compensation and age.

- Explore relationships between compensation, job satisfaction, and professional coding experience.

- Analyze data composition across developer roles and countries.

- Compare compensation across employment types and satisfaction levels.

Box plots provided clear insights into the spread, outliers, and central tendencies of various features in the dataset.


## Authors:
Ayushi Jain


### Other Contributors:
- Rav Ahuja
- Lakshmi Holla
- Malika


<!--## Change Log
|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|               
|2024-10-07|1.2|Madhusudan Moole|Reviewed and updated lab|                                                                                      
|2024-10-06|1.0|Raghul Ramesh|Created lab|-->


Copyright © IBM Corporation. All rights reserved.
